# Data Visualisation of COVID-19 in the UK using Sqlite, Pandas and Seaborn Libraries 
## Part 2a: Extract the Data

This is the second notebook in a 3-part series which explores the UK Gov's COVID-19 dashboard data which is publically available for download via a REST API from gov.uk.  We make use of a Sqlite3 database to query this data using SQL and import aggregations into Pandas data frames.  We then use the Seaborn library to visualise the results.

In this notebook (Part 2) - we first extract the data using a REST API based on the GOV.UK COVID-19 SDK and then insert the data into empty tables created in Part 1.  Part 3 will then query and visualise the data.

The data used in this notebook is publically available and more information can be found here:
https://coronavirus.data.gov.uk/details/about-data

### Configuration and Setup

The main library we will use to perform the REST API calls to get the GOV.UK COVID-19 data is the SDK published by GOV.UK.  This will handle the various calls, but also pagination as well and populate the data into a Pandas data-frame.

Install the library using `pip install uk-covid19`

For more information on the SDK, see: https://pypi.org/project/uk-covid19/

In [1]:
from uk_covid19 import Cov19API  # Use the UK COVID-19 GOV.UK SDK
import pandas as pd              # Pandas library for Dataframes
import time                      # Need the Sleep function for API calls
import csv                       # For CSV generation
from datetime import datetime    # Datetime functionality
import re                        # Regular Expression for the snake_case function

### Camel Case to Snake Case Function for Column Names

We're going to use a naming standard for the SQL tables based on Snake Case (lowercase and underscore only).  One reason we might do this is because it's more compliant with various database systems such as HiveQL, Amazon Athena, etc.

Unfortunately the column naming used by the GOV.UK API is based on Camel Case (mixed case with no spaces).  So the following function will be used to convert the source column names into snake case which will make importing the data easier.

In [2]:
def camel_to_snake(name: str) -> str:
    """
    Description: Convert any string to snake case (lower case and _ for spacing)
    Args:        name: input string to convert
    Returns:     input string converted to snake case 
    """   
    name = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    name = re.sub("[,.]", "_", name)   
    name = re.sub("[+*&%=()?<>!@#$/\\\\]", "", name)  

    return re.sub("([a-z0-9])([A-Z])", r"\1_\2", name).lower().replace(" ","_").replace("-","_").replace("__","_").replace("__","_").replace("deaths28","deaths_28")   

### Handling Metrics and Area Types in the REST API

If you read the documentation provided at https://coronavirus.data.gov.uk/details/download and https://coronavirus.data.gov.uk/details/developers-guide - you'll see that only a maximum of 5 metrics can be requested via the API in addition to the standard (primary key) metrics: Area Type, Name, Code and Date.

This means we have to make multiple API requests for groups of metrics since we can't request all in one go.  Additionally the metrics available differ depending on the Area Type you are requesting.

The following provides a configuration in the form of a dictionary object which define the metrics available for each area type.  In addition the metrics are broken down into groups of no more than 5 metrics and this will allow us to iterate in a more generic function.  The default metrics are also provided in a separate list object.

This notebook is going to retrieve data for all area types, and for most of the metrics available.  However, this is forever changing so as more metrics become available we simply add them to one or more separate metric groups for the appropriate area type.

In [3]:
default_metrics = ["areaType",
                   "areaName",
                   "areaCode",
                   "date"]

metrics_by_area_type = {
    "overview" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","newPillarOneTestsByPublishDate","cumPillarOneTestsByPublishDate"],
        ["newPillarTwoTestsByPublishDate","cumPillarTwoTestsByPublishDate","newPillarThreeTestsByPublishDate","cumPillarThreeTestsByPublishDate","newPillarFourTestsByPublishDate"],
        ["cumPillarFourTestsByPublishDate","newAdmissions","cumAdmissions","cumTestsByPublishDate"],
        ["newTestsByPublishDate","covidOccupiedMVBeds","hospitalCases","plannedCapacityByPublishDate","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ],
    "nation" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","maleCases","femaleCases","newPillarOneTestsByPublishDate","cumPillarOneTestsByPublishDate"],
        ["newPillarTwoTestsByPublishDate","cumPillarTwoTestsByPublishDate","newPillarThreeTestsByPublishDate","cumPillarThreeTestsByPublishDate"],
        ["newAdmissions","cumAdmissions","cumAdmissionsByAge","cumTestsByPublishDate"],
        ["newTestsByPublishDate","covidOccupiedMVBeds","hospitalCases","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"],
        ["newPeopleVaccinatedFirstDoseByPublishDate","newPeopleVaccinatedSecondDoseByPublishDate","cumPeopleVaccinatedFirstDoseByPublishDate","cumPeopleVaccinatedSecondDoseByPublishDate"],
        ["newPeopleVaccinatedThirdInjectionByPublishDate","cumPeopleVaccinatedThirdInjectionByPublishDate"]
    ],
    "region" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","maleCases","femaleCases","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ],  
    "nhsRegion" : [
        ["newAdmissions","cumAdmissions","cumAdmissionsByAge","covidOccupiedMVBeds","hospitalCases"]
    ],  
    "utla" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ],
    "ltla" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ]    
}

### Extract the Data as CSV by iterating through the metric groups

Now we simply iterate through each area type which provides a list of metric groups.  Then we iterate through each metric group (no more than 5 metrics) and combined with the mandatory default metrics we use the SDK to make a REST API call and grab the data into a Pandas dataframe.

Unfortunately there is rate limiting implemented for this API as documentated in the fair usage policy: https://coronavirus.data.gov.uk/details/download.  As such we use `time.sleep(1)` to pause for one second between each API call.  So far this has provded sufficient to abide by the fair usage policy.

As we iterate through each metric group, we combine all the data together using an outer join merge based on the default metric columns.

We then convert the column names to snake case, convert the `date` column to datetime, and finally write the data out to a CSV (so one CSV per area type is generated).

**So why CSV and not load straight to the database?** 

It's common in most ETL/ELT processing systems to initially stage the data before loading into a database - typically to a data-lake of some sort.  Here we simply use the file system and stage the data as CSV.  This means if we have any errors loading the data into the database tables, we have the data in a ready to use format to investigate the issues further.  

**NOTE:** The following code will take several minutes to complete.

In [4]:
# iterate through each area type (this provides a list of metric groups)
# key = areaType, value = list of metric groups (or a list of lists of metrics)
for key, value in metrics_by_area_type.items():
    
    filters = [f"areaType={key}"]
    
    df_merged = pd.DataFrame()
    
    # iterate through each metric group (at most 5 metrics)
    metric_group_index = 1
    for metric_group in value:
        # we create a dictionary containing the metrics which must also include the default metrics
        structure = {}
        structure_d = {k: k for k in default_metrics}
        structure_m = {k: k for k in metric_group}
        structure.update(structure_d)
        structure.update(structure_m)

        # call the API using the SDK - requires just the Area Type and Metrics requested 
        api = Cov19API(
            filters=filters,
            structure=structure
        ) 
        
        # grab data as a Pandas dataframe and merge with any previously requested metrics
        df = api.get_dataframe() 
        print(f"{datetime.now().strftime('%H:%M:%S')} : Area Type: {key} - Metric Group {str(metric_group_index)} - Num records: {len(df)}")
        if df_merged.empty:
            df_merged = df
        else:
            df_merged = df_merged.merge(df, on=default_metrics, how="outer")
            
        # pause for a second to avoid breaching the fair usage policy
        time.sleep(1.0)
        metric_group_index += 1
     
    # all the metrics have now been retrieved and merged so convert columns to snake_case and write out the area type's CSV data
    df_merged.rename(columns=camel_to_snake, inplace=True)
    df_merged["date"]= pd.to_datetime(df_merged["date"])
    df_merged.to_csv(f"{key}.csv", index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

14:39:54 : Area Type: overview - Metric Group 1 - Num records: 716
14:39:55 : Area Type: overview - Metric Group 2 - Num records: 686
14:39:57 : Area Type: overview - Metric Group 3 - Num records: 654
14:39:58 : Area Type: overview - Metric Group 4 - Num records: 662
14:40:00 : Area Type: overview - Metric Group 5 - Num records: 680
14:40:01 : Area Type: overview - Metric Group 6 - Num records: 684
14:40:03 : Area Type: nation - Metric Group 1 - Num records: 2772
14:40:05 : Area Type: nation - Metric Group 2 - Num records: 2760
14:40:07 : Area Type: nation - Metric Group 3 - Num records: 2353
14:40:09 : Area Type: nation - Metric Group 4 - Num records: 2695
14:40:11 : Area Type: nation - Metric Group 5 - Num records: 2706
14:40:13 : Area Type: nation - Metric Group 6 - Num records: 2696
14:40:15 : Area Type: nation - Metric Group 7 - Num records: 1476
14:40:16 : Area Type: nation - Metric Group 8 - Num records: 390
14:40:19 : Area Type: region - Metric Group 1 - Num records: 6258
14:40

### Supplementary Downloads

We need to download population data for the various geographies (such as UTLA) which are available from GOV.UK.  This will help provide standardised case and death rates between different areas.  

As was the case for the COVID-19 data, we snake case the column names.

In [5]:
df_populations = pd.read_csv(f"https://coronavirus.data.gov.uk/downloads/supplements/ONS-population_2021-08-05.csv")
df_populations.rename(columns=camel_to_snake, inplace=True)
df_populations.to_csv(f"populations.csv", index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

# preview the data
df_populations.head()

,category,area_code,gender,age,population
0,AGE_ONLY,E06000001,ALL,00_04,5147
1,AGE_ONLY,E06000001,ALL,00_59,69434
2,AGE_ONLY,E06000001,ALL,05_09,5774
3,AGE_ONLY,E06000001,ALL,10_14,5982
4,AGE_ONLY,E06000001,ALL,15_19,5126


### Generating Age/Gender Data from JSON based fields

You may have noticed in some of the data files generated (like `nation.csv`) that there are some fields that contain a JSON representation of age bracketed data.  For example, the `male_cases` field contains data that looks like this:

`[{'age': '30_to_34', 'rate': 12870.5, 'value': 246652}, {'age': '35_to_39', 'rate': 11484.5, 'value': 212805}]`

We'd like to parse this data and convert it into more useable data - i.e. have a data-set which provides case numbers by age (group) and gender in addition to the existing key fields (area and date).

Sqlite doesn't have native support for JSON parsing.  Although there are extensions available, this notebook is going to just use the standard Sqlite database and library.  Therefore we're going to transform this data using Pandas Dataframes - first we'll load the `nation.csv` data, perform the transformation and write out the data as a CSV.  Then in the next part of this notebook, we'll load all the CSVs into the Sqlite database.

Most of the parsing of JSON is done using the `pd.concat` and `eval` functions as shown below.  Before we do this, we combine the male cases data with the female cases data and derive an additional gender field.

In [6]:
# read in the full national level data
df = pd.read_csv("nation.csv")

# generate male cases sub-set.  Note we want to filter out all the NaNs, empty strings, and empty lists [] since
# these cause issues when using the pd.DataFrame constructor - so filter all records where the JSON is 2 characters or less
df_cases_m = df.loc[df["male_cases"].str.len()>2, ["area_type","area_name","area_code","date","male_cases"]].rename(columns={"male_cases": "cases"})
df_cases_m["gender"] = "male"

# now repeat the same process for female cases
df_cases_f = df.loc[df["female_cases"].str.len()>2,["area_type","area_name","area_code","date","female_cases"]].rename(columns={"female_cases": "cases"})
df_cases_f["gender"] = "female"

# now combine the two and replace with a new index
df_cases_mf = pd.concat([df_cases_m, df_cases_f], ignore_index=True, sort=False)

# now we expand the JSON to create additional records/fields and use the index as the key
# this will have a column called 'index' which we can use to join on to df_cases_mf to get the area/gender/date fields
df_cases_by_age_index = pd.concat([pd.DataFrame(eval(x)) for x in df_cases_mf["cases"]], keys=df_cases_mf.index).reset_index(level=1, drop=True).reset_index()

# now we need to use the index to join onto the previous data frame in order to add the key fields (area, date and gender)
df_cases_by_age = df_cases_mf[["area_type","area_name","area_code","date","gender"]].merge(df_cases_by_age_index, left_index=True, right_on="index")

# drop the index column
df_cases_by_age = df_cases_by_age.drop(columns=["index"])

# currently the "value" column is cumulative cases, but we'd like new cases only so we need to 
# sort by date, do a group by and then perform a different on the value column
df_cases_by_age = df_cases_by_age.sort_values(by=["area_type","area_name","area_code","gender","age","date"])
df_cases_by_age["new_cases"] = df_cases_by_age.groupby(["area_type","area_name","area_code","gender","age"])["value"].diff().fillna(0)

# rename the values column
df_cases_by_age = df_cases_by_age.rename(columns={"value": "cum_cases"})

# finally write the cases by ages data out to CSV
df_cases_by_age.to_csv(f"cases.csv", index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

# preview the data
df_cases_by_age.tail()

,area_type,area_name,area_code,date,gender,age,rate,cum_cases,new_cases
13280,nation,Wales,W92000004,2022-01-13,male,80_to_84,11476.0,4623,0.0
13258,nation,Wales,W92000004,2022-01-13,male,85_to_89,13363.6,2911,0.0
13277,nation,Wales,W92000004,2022-01-13,male,85_to_89,13363.6,2911,0.0
13255,nation,Wales,W92000004,2022-01-13,male,90+,19134.6,1897,0.0
13271,nation,Wales,W92000004,2022-01-13,male,90+,19134.6,1897,0.0


## Part 2b: Load the Data

Now we have the data staged a set of CSV files.  All that is left to load these in to the sqlite database that was created by the first notebook.

In [7]:
import sqlite3
sqlite_db_path = "c19.db"

### Optional: Uncompress Sqlite database file 

If you compressed the database in the previous notebook, then you'll need to first decompress the file before opening it.

In [8]:
import os
import gzip
import shutil

with gzip.open(sqlite_db_path + '.gz', 'rb') as f_in:
    with open(sqlite_db_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

### Open the database connection

In [9]:
conn = sqlite3.connect(sqlite_db_path)

### Delete any existing data

Although the Pandas function `to_sql` supports 'replace' for the `if_exists` parameter - it does this by dropping and re-creating the table.  Unfortunately doing this loses the original data types.  

Normally this wouldn't be important but if the `date` column is not a `DATE` data-type then the Seaborn plots (in the next notebook) do not handle the X-axis labels gracefully.

Therefore, we first delete the data in each of the tables, and then perform the insert using 'append' for the `if_exists` parameter (since there should be no data this is safe).  Using the value 'fail' would also not work since the tables already exist - so append is the only way we can insert the data into an existing table and retain the data-types.

In [10]:
sql = """
DELETE FROM c19dashboard_uk__national_daily_metrics;
DELETE FROM c19dashboard_uk__summary_daily_metrics;
DELETE FROM c19dashboard_uk__nhsregion_daily_metrics;
DELETE FROM c19dashboard_uk__region_daily_metrics;
DELETE FROM c19dashboard_uk__utla_daily_metrics;
DELETE FROM c19dashboard_uk__ltla_daily_metrics;
DELETE FROM c19dashboard_uk__national_cases_by_age_gender;
DELETE FROM reference_geography__age_gender_populations;
DELETE FROM reference_geography__ltla_utla_region_mappings;
"""
_ = conn.executescript(sql)

### Load the CSV files into the database

Now we simply load the data into the database using intermediary Pandas dataframes.  Note this assumes the tables are currently empty - the CSV file is appended to the tables.

In [11]:
pd.read_csv(f"nation.csv").to_sql("c19dashboard_uk__national_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"overview.csv").to_sql("c19dashboard_uk__summary_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"nhsRegion.csv").to_sql("c19dashboard_uk__nhsregion_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"region.csv").to_sql("c19dashboard_uk__region_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"utla.csv").to_sql("c19dashboard_uk__utla_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"ltla.csv").to_sql("c19dashboard_uk__ltla_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"cases.csv").to_sql("c19dashboard_uk__national_cases_by_age_gender", conn, if_exists="append", index=False)
pd.read_csv(f"populations.csv").to_sql("reference_geography__age_gender_populations", conn, if_exists="append", index=False)
pd.read_csv(f"ltla_utla_region_mappings.csv").to_sql("reference_geography__ltla_utla_region_mappings", conn, if_exists="append", index=False)

### Check Row Counts

We can run a quick query against the Sqlite database to check the row counts for each of the tables to ensure the data has loaded.

In [12]:
sql = """
SELECT 'c19dashboard_uk__national_daily_metrics' AS [table], COUNT(*) AS rows FROM c19dashboard_uk__national_daily_metrics
UNION ALL 
SELECT 'c19dashboard_uk__summary_daily_metrics' AS [table], COUNT(*) AS rows FROM c19dashboard_uk__summary_daily_metrics
UNION ALL 
SELECT 'c19dashboard_uk__nhsregion_daily_metrics' AS [table], COUNT(*) AS rows FROM c19dashboard_uk__nhsregion_daily_metrics
UNION ALL 
SELECT 'c19dashboard_uk__region_daily_metrics' AS [table], COUNT(*) AS rows FROM c19dashboard_uk__region_daily_metrics
UNION ALL 
SELECT 'c19dashboard_uk__utla_daily_metrics' AS [table], COUNT(*) AS rows FROM c19dashboard_uk__utla_daily_metrics
UNION ALL 
SELECT 'c19dashboard_uk__ltla_daily_metrics' AS [table], COUNT(*) AS rows FROM c19dashboard_uk__ltla_daily_metrics
UNION ALL 
SELECT 'c19dashboard_uk__national_cases_by_age_gender' AS [table], COUNT(*) AS rows FROM c19dashboard_uk__national_cases_by_age_gender
UNION ALL 
SELECT 'reference_geography__age_gender_populations' AS [table], COUNT(*) AS rows FROM reference_geography__age_gender_populations
UNION ALL 
SELECT 'reference_geography__ltla_utla_region_mappings' AS [table], COUNT(*) AS rows FROM reference_geography__ltla_utla_region_mappings
"""
pd.read_sql(sql, conn, index_col="table")

,rows
table,
c19dashboard_uk__national_daily_metrics,2776
c19dashboard_uk__summary_daily_metrics,716
c19dashboard_uk__nhsregion_daily_metrics,4670
c19dashboard_uk__region_daily_metrics,6260
c19dashboard_uk__utla_daily_metrics,145626
c19dashboard_uk__ltla_daily_metrics,257879
c19dashboard_uk__national_cases_by_age_gender,26562
reference_geography__age_gender_populations,10851
reference_geography__ltla_utla_region_mappings,336


### Cleanup

Ensure all changes are committed and then close the Sqlite connection.  Also force garbage collection - at this point, there should be no locks on the database file so it could be zipped up and deleted.

In [13]:
import gc

# Commit and close Sqlite connection
conn.commit()
conn.close()

# Force garbage collection
_ = gc.collect(2)

### Optional: Compress Sqlite database file 

To help keep file sizes small and to allow the database to be easily stored in a Git repo, you may want to compress the database file and then remove the original.

In [14]:
import os
import gzip
import shutil

with open(sqlite_db_path, "rb") as f_in:
    with gzip.open(sqlite_db_path + ".gz", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
        
os.remove(sqlite_db_path)